In [52]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import string
import nltk
import string
import numpy as np
import json
import pandas as pd
import math

from nltk.corpus import reuters, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def get_user_tweets(user):

    url = f'https://twitter.com/{user}'

    # Define the options for the Chrome webdriver
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--incognito")
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument("--enable-javascript")

    driver = webdriver.Chrome(options=options)

    # Load the page
    driver.get(url)

    # Wait for tweets to populate the page
    try:
        WebDriverWait(driver, 5).until(expected_conditions.presence_of_element_located(
            (By.CSS_SELECTOR, '[data-testid="tweet"]')))
    except WebDriverException:
        return

    # Extract the HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    posts = soup.find_all(attrs={"data-testid": "tweetText"})

    # Extract the text content of each post
    post_content = [post.text for post in posts]

    # Save the data in a CSV file
    data = pd.DataFrame(post_content)

    driver.quit()

    return(data)

def get_tweets():
    df = pd.read_csv('./data/users.csv', delimiter=',')

    tweets = pd.DataFrame()

    for i in range(0, len(df)):
        if i % 10 == 0:
            print(i,'/',len(df)) 

        tweets = get_user_tweets(df['Twitter_username'][i])

        try:
            tweets.to_csv('./data/twitter_posts.csv', mode='a',index=False, header=False)
        except:
            continue

def clean_text(text_to_clean):
    try:
        punctuation_list = list(string.punctuation)
        punctuation_list.append('\'\'')
        punctuation_list.append('``')
        punctuation_list.append('--')
        punctuation_list.append('...')

        text_to_clean['text'] = word_tokenize(text_to_clean['text'])

        text_to_clean['text'] = [word for word in text_to_clean['text'] if word not in stopwords.words('english')]

        text_to_clean['text'] = [word for word in text_to_clean['text'] if word not in punctuation_list]

        text_to_clean['text'] = [word.lower() for word in text_to_clean['text']]
        
        return  text_to_clean['text']
    except Exception as e:
        print(f'Failed with: {text_to_clean}, {e}')

In [58]:
tweets = pd.read_csv('./data//twitter_posts.csv', delimiter=',', names=['text'], header=None)
tweets

,text
0,"Ngoc Pham, a 83 yr old Vietnamese man, was one..."
1,I’m making the following announcement and form...
2,“The most aggressive field operation in Califo...
3,Thank you Supervisor @AaronPeskin for endorsin...
4,#tbt sharing a meal w/my friend @narendramodi ...
...,...
1823,Unborn children should be welcomed in life and...
1824,I will not vote for John Boehner. My statemen...
1825,I believe that warriors like Chris Kyle are so...
1826,Millions of Americans stand with #Israel. RT ...


In [59]:
tweets = tweets.dropna()
count_nan = tweets['text'].isnull().sum()
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: 0


In [61]:
# ~20 seconds
tweets.apply(clean_text, axis=1)
tweets

,text
0,"[ngoc, pham, 83, yr, old, vietnamese, man, one..."
1,"[i, ’, making, following, announcement, formal..."
2,"[“, the, aggressive, field, operation, califor..."
3,"[thank, supervisor, aaronpeskin, endorsing, se..."
4,"[tbt, sharing, meal, w/my, friend, narendramod..."
...,...
1823,"[unborn, children, welcomed, life, protected, ..."
1824,"[i, vote, john, boehner, my, statement, http, ..."
1825,"[i, believe, warriors, like, chris, kyle, best..."
1826,"[millions, americans, stand, israel, rt, istan..."


In [62]:
# 10 - 30 seconds
print(get_user_tweets('realdonaldtrump'))

                                                   0
0  Tonight, @FLOTUS and I tested positive for COV...
1     Going welI, I think! Thank you to all. LOVE!!!
2                     I WON THIS ELECTION, BY A LOT!
